In [38]:
from nipype.interfaces import cat12
from nipype.interfaces import spm
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
import os
import os.path

In [34]:
def construct_pipe(nii):
    seg = pe.Node(interface=cat12.CAT12Segment(), name = 'seg')
    seg.inputs.in_files = nii
    smooth = pe.Node(interface=spm.Smooth(), name = "smooth")
    smooth.inputs.fwhm = 4
    preprocessing = pe.Workflow(name="pre"+nii[-5:-4], base_dir=os.getcwd()+'/test/')
    preprocessing.connect([(seg, smooth, [("gm_modulated_image", "in_files")])])
    return preprocessing

In [35]:
abspath = os.path.realpath('../test')
mr_list = [os.path.join(abspath, 'test1.nii'), os.path.join(abspath, 'test2.nii')]
pipes = list(map(construct_pipe, mr_list))

In [51]:
datasrc = pe.Node(nio.DataGrabber(infields=['subject_id'], outfields=['struct']), name='grab')
datasrc.inputs.base_directory = abspath
datasrc.inputs.sort_filelist = False
datasrc.inputs.subject_id = [1, 2]
datasrc.inputs.template = 'test%d.nii'
ds = pe.Node(nio.DataSink(), name='sink')
ds.inputs.base_directory = abspath
seg = pe.Node(interface=cat12.CAT12Segment(), name = 'seg')
smooth = pe.Node(interface=spm.Smooth(), name = "smooth")
smooth.inputs.fwhm = 4
preprocessing = pe.Workflow(name="preprocessing", base_dir=os.getcwd()+'/test/')
preprocessing.connect([
    (seg, smooth, [("gm_modulated_image", "in_files")]),
    (datasrc, seg, [("struct", "in_files")]),
    #(datasrc, ds, [("subject_id", "container")]),
    (seg, ds, [("gm_modulated_image", "gm")]),
    (smooth, ds, [("smoothed_files", "sgm")])
    ])

In [52]:
preprocessing.run()

230130-16:20:43,148 nipype.workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
230130-16:20:43,153 nipype.workflow INFO:
	 Running serially.
230130-16:20:43,154 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.grab" in "/home/biobot/levodopa/workspace/test/preprocessing/grab".
230130-16:20:43,156 nipype.workflow INFO:
	 [Node] Executing "grab" <nipype.interfaces.io.DataGrabber>
230130-16:20:43,157 nipype.workflow INFO:
	 [Node] Finished "grab", elapsed time 0.000198s.
230130-16:20:43,159 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.seg" in "/home/biobot/levodopa/workspace/test/preprocessing/seg".
230130-16:20:43,163 nipype.workflow INFO:
	 [Node] Executing "seg" <nipype.interfaces.cat12.preprocess.CAT12Segment>
230130-17:02:08,582 nipype.workflow INFO:
	 [Node] Finished "seg", elapsed time 2485.418241s.
230130-17:02:08,600 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.smooth" in "/home/biobot/levodopa/workspac

stty: 标准输入: 对设备不适当的 ioctl 操作


230130-17:02:20,404 nipype.workflow INFO:
	 [Node] Finished "smooth", elapsed time 11.801266s.
230130-17:02:20,407 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.sink" in "/home/biobot/levodopa/workspace/test/preprocessing/sink".
230130-17:02:20,410 nipype.workflow INFO:
	 [Node] Executing "sink" <nipype.interfaces.io.DataSink>
230130-17:02:20,412 nipype.workflow INFO:
	 [Node] Finished "sink", elapsed time 0.00068s.


stty: 标准输入: 对设备不适当的 ioctl 操作
